# 구매 후기 긍정 / 부정 예측하기

## 데이터 불러오기

In [1]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
from sklearn.model_selection import train_test_split

In [2]:
# 경고창 무시
import warnings
warnings.filterwarnings(action='ignore')

In [3]:
review = pd.read_table('./ratings_total.txt', names=['review_score', 'review_content'])
review = review[:40]
review[:3]

,review_score,review_content
0,5,배송 빠르고 굿
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
2,5,아주 좋아요 바지 정말 좋아서2개 더 구매했어요 이 가격에 대박입니다. 바느질이 조...


## 데이터 정제

In [4]:
review['review_content'] = review['review_content'].apply(lambda x: re.sub(r'[^ ㄱ-ㅣ가-힣]+', '', x))
review['review_content'][:5]

0                                             배송 빠르고 굿
1                        택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
2    아주 좋아요 바지 정말 좋아서개 더 구매했어요 이 가격에 대박입니다 바느질이 조금 ...
3    선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다 전화...
4                    민트 색상 예뻐요 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ
Name: review_content, dtype: object

In [5]:
review.drop_duplicates(subset=['review_content'], inplace=True)

In [6]:
review['review_content'][:3]

0                                             배송 빠르고 굿
1                        택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
2    아주 좋아요 바지 정말 좋아서개 더 구매했어요 이 가격에 대박입니다 바느질이 조금 ...
Name: review_content, dtype: object

## label 생성 --> 긍정/부정 사전에서 긍정/부정 분류하기

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [8]:
positive_dict = ['빠르고', '굿', '더', '좋아', '정말', '대박', '빨리', '예뻐요', '예쁘', '실용', '딱', '귀엽', '넘', '저렴',
                 '씁', '좋고', '좋다', '좋아', '편한', '편함', '편하다', '재구매', '구매', '가성비', 
                 '젤', '빠른', '배송', '빠', '여유', '마음',
                 '최고', '추가', '좋은', '감사', '신용', '귀엽고', '생각보다', '저렴한', '모두', '맘에', '들다', '맘에는', '좋아요', 'ㅎㅎ']
negative_dict = ['엉망', '도', '없이', '가고', '조금', '만', '비추', '불편', '별로',
                 '안', '때', '작은', '빨', '비싼', '별루',
                 '몰랐', '이상', '오랫', '않아요', '불친절', '불쾌', '최악', '떨어짐',
                 '빠지', '없', '높아', '근데', '따로', '번졌', '않',
                 '깔끄', '의심', '폭망', '떡', '걱정']

In [9]:
tfidf1 = TfidfVectorizer()
tfidf2 = TfidfVectorizer()

In [10]:
tfidf1.fit_transform(positive_dict).toarray()
tfidf2.fit_transform(negative_dict).toarray()

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],


In [11]:
# positive_words_count=tfidf1.transform([review['review_content'][0]]).toarray().sum()
# negative_words_count=tfidf2.transform([review['review_content'][0]]).toarray().sum()

In [12]:
# review.loc[0]

In [13]:
# review['label'] = np.nan

In [14]:
# if positive_words_count - negative_words_count > 0:
#     review.at[0, 'label'] = 1
# else:
#     review.at[0, 'label'] = 0
# print(review.loc[0])

## 긍정인지 부정인지 판단해서 라벨 값 생성 (1 : 긍정, 0 : 부정)

In [15]:
def pos_neg(df, column):
    df['label'] = np.nan
    count = len(df)
    for i in range(count):
        print('--------------------'+ str(i) +'--------------------')
        positive_words_count=tfidf1.transform([df[column][i]]).toarray().sum()
        negative_words_count=tfidf2.transform([df[column][i]]).toarray().sum()
        if positive_words_count - negative_words_count > 0:
            df.at[i, 'label'] = 1
        else:
            df.at[i, 'label'] = 0
        print(df.loc[i])

In [16]:
pos_neg(review, 'review_content')

--------------------0--------------------
review_score             5
review_content    배송 빠르고 굿
label                    1
Name: 0, dtype: object
--------------------1--------------------
review_score                                  2
review_content    택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
label                                         0
Name: 1, dtype: object
--------------------2--------------------
review_score                                                      5
review_content    아주 좋아요 바지 정말 좋아서개 더 구매했어요 이 가격에 대박입니다 바느질이 조금 ...
label                                                             1
Name: 2, dtype: object
--------------------3--------------------
review_score                                                      2
review_content    선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다 전화...
label                                                             1
Name: 3, dtype: object
--------------------4--------------------
review_score                                      5
review_conten

In [17]:
review

,review_score,review_content,label
0,5,배송 빠르고 굿,1.0
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고,0.0
2,5,아주 좋아요 바지 정말 좋아서개 더 구매했어요 이 가격에 대박입니다 바느질이 조금 ...,1.0
3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다 전화...,1.0
4,5,민트 색상 예뻐요 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ,1.0
5,2,비추합니다 계란 뒤집을 때 완전 불편해요 ㅠㅠ 코팅도 묻어나고 보기엔 예쁘고 실용적...,1.0
6,1,주문을 월 에 시켰는데 월 일에 배송이 왔네요 ㅎㅎㅎ 여기 회사 측과는 전화도 안되...,0.0
7,2,넉넉한 길이로 주문했는데도 안 맞네요 별로예요,0.0
8,2,보폴이 계속 때처럼 나오다가 지금은 안나네요,0.0
9,2,인데 전문 속옷 브랜드 위생 팬티 보다 작은 듯해요 불편해요 밴딩부분이 다 신축성 ...,0.0


In [20]:
# train data csv로 저장
review.to_csv('train_df.csv', header=False, index=False)

### >> RANKS NL에서 제공하는 한국어 불용어 사전 사용  

In [18]:
# from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [19]:
# stopwords = pd.read_csv("https://raw.githubusercontent.com/yoonkt200/FastCampusDataset/master/korean_stopwords.txt").values.tolist()
# stopwords = sum(stopwords, [])
# stopwords[:10]

['휴', '아이구', '아이쿠', '아이고', '어', '나', '우리', '저희', '따라', '의해']